# Pet Dataset Segmentation
## Machine Learning data mining final project
### __Team members:__ Vedanshi Shah, Vedant Bhagat, Sid Mallareddygari

In [ ]:
# imports


# Set up
#### Describe the dataset, including its basic statistics. 